In [1]:
import numpy as np

def pre_sampling(params):
    params[0] = np.random.normal(0, 1, 1)
    return params

def post_sampling(params, m):
    params[1] = np.random.normal(0, 1, m)
    return params

def calc(params, scr):
    return scr * (params[0] + params[1])

def evppi_calc(result, level, m):
    params = np.zeros((2, m))
    params = pre_sampling(params)
    params = post_sampling(params, m)
    
    if level == 0:
        sum_of_max = 0
        sum_of_no_scr = 0
        sum_of_scr = 0
        
        for i in range(m):
            no_scr = calc(params[:, i], scr = 0)
            scr = calc(params[:, i], scr = 1)
            
            sum_of_max += max(no_scr, scr)
            sum_of_no_scr += no_scr
            sum_of_scr += scr
            
        sum_of_max /= m
        max_of_sum = max(sum_of_no_scr, sum_of_scr) / m
        p = sum_of_max - max_of_sum
        
        result[level][0] += p
        result[level][1] += p * p
        result[level][2] += p
        result[level][3] += p * p
        result[level][4] += p * p * p
        result[level][5] += p * p * p * p
    
    else:
        sum_of_max = 0
        sum_of_no_scr_first = 0
        sum_of_scr_first = 0
        sum_of_no_scr_second = 0
        sum_of_scr_second = 0
        
        for i in range(int(m / 2)):
            no_scr = calc(params[:, i], scr = 0)
            scr = calc(params[:, i], scr = 1)
            
            sum_of_max += max(no_scr, scr)
            sum_of_no_scr_first += no_scr
            sum_of_scr_first += scr
            
        for i in range(int(m / 2), m):
            no_scr = calc(params[:, i], scr = 0)
            scr = calc(params[:, i], scr = 1)
            
            sum_of_max += max(no_scr, scr)
            sum_of_no_scr_second += no_scr
            sum_of_scr_second += scr
            
        sum_of_max /= m
        max_of_sum = max(sum_of_no_scr_first + sum_of_no_scr_second, sum_of_scr_first + sum_of_scr_second) / m
        p = sum_of_max - max_of_sum
        result[level][0] += p
        result[level][1] += p * p
        
        max_of_sum_first = max(sum_of_no_scr_first, sum_of_scr_first) / int(m / 2)
        max_of_sum_second = max(sum_of_no_scr_second, sum_of_scr_second) / (m - int(m / 2))
        z = (max_of_sum_first + max_of_sum_second) / 2 - max_of_sum
        result[level][2] += z
        result[level][3] += z * z
        result[level][4] += z * z * z
        result[level][5] += z * z * z * z
        
    return result
    
    
def mlmc_calc(level, m, n_samples, n_samples_done):
    result = np.zeros((11, 6))
    for l in range(level + 1):
        for _ in range(n_samples[l] - n_samples_done[l]):
            result = evppi_calc(result, l, m)
        n_samples_done[l] = n_samples[l]
        
    return result
        

def mlmc_test():
    print('l aveZ aveP varZ varP kurt check')
    print('-----------------------------')
    
    m = 1
    n_samples = [2000 for _ in range(11)]
    n_samples_done = [0 for _ in range(11)]
    for level in range(11):
        result = mlmc_calc(level, m, n_samples, n_samples_done)
        m *= 2
        
        n = n_samples[level]
        aveZ = result[level][2] / n
        aveP = result[level][0] / n
        varZ = result[level][3] / n - aveZ * aveZ
        varP = result[level][1] / n - aveP * aveP
        kurt = 0
        if level > 0:
            kurt = (result[level][5] / n - 4 * result[level][4] / n * aveZ + 6 * result[level][3] / n * aveZ ** 2 - 3 * aveZ ** 4) / varZ ** 2

        print(str(level) + ' ' + str(aveZ) + ' ' + str(aveP) + ' ' + str(varZ) + ' ' + str(varP) + ' ' + str(kurt))
        
mlmc_test()

l aveZ aveP varZ varP kurt check
-----------------------------
0 0.0 0.0 0.0 0.0 0
1 0.07848238977686614 0.07848238977686614 0.025135265267406323 0.025135265267406323 8.991617971052749
2 0.04230877259793777 0.1135214485781194 0.009390769133067467 0.02029185508676579 10.994397739602169
3 0.022284067265391044 0.13761458055676276 0.0038502704633128666 0.01826646390889967 17.076953286886628
4 0.012831185207149712 0.15600482883616135 0.0016247859830370594 0.01645049976056321 21.413054511440446
5 0.006021368190771662 0.16044366552196504 0.0006108546766810663 0.014737811636851495 34.077843795651106
6 0.002313637324354636 0.15695991742645699 0.00013989862377176792 0.013570385462321765 49.71626675479096
7 0.0014653123628564745 0.16006352323824827 6.381467292708567e-05 0.013783671542957301 65.53248177990949
8 0.000930007813268932 0.16396955858611187 2.978635887400716e-05 0.013659296911638093 73.70795268693477
9 0.0004476489022915103 0.16263384216674756 1.1171719291680236e-05 0.013354008063274754